# model code

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoImageProcessor, ResNetForImageClassification
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset, Subset
import numpy as np
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

In [18]:
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
base_model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")


class CustomResNetModel(nn.Module):
    def __init__(self, base_model, num_classes=3):
        super(CustomResNetModel, self).__init__()
        # Freeze the base ResNet model
        for param in base_model.parameters():
            param.requires_grad = False
        
        self.resnet = base_model 
        self.dropout1 = nn.Dropout(p=0.5)
        self.dense1 = nn.Linear(1000, 512) 
        self.dense2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(p=0.5)
        self.dense3 = nn.Linear(256, 128)
        self.output = nn.Linear(128, num_classes) 

    def forward(self, x):
        x = self.resnet(x).logits 
        x = self.dropout1(x)
        x = torch.relu(self.dense1(x))
        x = torch.relu(self.dense2(x))
        x = self.dropout2(x)
        x = torch.relu(self.dense3(x))
        x = self.output(x) 
        return x

num_classes = 2
custom_model = CustomResNetModel(base_model, num_classes=num_classes)


In [19]:
print(custom_model)

CustomResNetModel(
  (resnet): ResNetForImageClassification(
    (resnet): ResNetModel(
      (embedder): ResNetEmbeddings(
        (embedder): ResNetConvLayer(
          (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
        )
        (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      )
      (encoder): ResNetEncoder(
        (stages): ModuleList(
          (0): ResNetStage(
            (layers): Sequential(
              (0): ResNetBottleNeckLayer(
                (shortcut): ResNetShortCut(
                  (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                )
                (layer): Sequential(
           

In [20]:
for name, param in custom_model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

resnet.resnet.embedder.embedder.convolution.weight: Frozen
resnet.resnet.embedder.embedder.normalization.weight: Frozen
resnet.resnet.embedder.embedder.normalization.bias: Frozen
resnet.resnet.encoder.stages.0.layers.0.shortcut.convolution.weight: Frozen
resnet.resnet.encoder.stages.0.layers.0.shortcut.normalization.weight: Frozen
resnet.resnet.encoder.stages.0.layers.0.shortcut.normalization.bias: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.0.convolution.weight: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.0.normalization.weight: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.0.normalization.bias: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.1.convolution.weight: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.1.normalization.weight: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.1.normalization.bias: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.2.convolution.weight: Frozen
resnet.resnet.encoder.stages.0.layers.0.layer.2.normalization.

# training code

In [23]:
dataset_path = "../nih_dataset/test_data"

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [24]:
print(dataset.class_to_idx)

{'normal': 0, 'pneumonia': 1}


In [25]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Show one batch
for batch in dataloader:
    X_batch, y_batch = batch
    print(f"Batch X shape: {X_batch.shape}, Batch y shape: {y_batch.shape}")
    break 

Batch X shape: torch.Size([32, 3, 1024, 1024]), Batch y shape: torch.Size([32])


In [26]:
# plt.figure(figsize=(10, 5))

# for i in range(10):
#     plt.subplot(1, 10, i + 1)  
#     plt.imshow(X[i]) 
#     plt.title(f"Label: {y[i]}")
#     plt.axis('off') 

# plt.tight_layout()
# plt.show()

In [27]:
# plt.figure(figsize=(10, 5))      

# for idx, i in enumerate(range(len(y) - 11, len(y) - 1)):
#     plt.subplot(1, 10, idx + 1)  
#     plt.imshow(X[i]) 
#     plt.title(f"Label: {y[i]}")
#     plt.axis('off')

# plt.tight_layout()
# plt.show()

In [32]:
y = np.array([label for _, label in dataset]) 

In [37]:
outer_folds = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
inner_folds = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Hyperparameter search space
learning_rates = [0.001, 0.0005, 0.0001]
batch_sizes = [16, 32]

outer_results = []

for train_idx, test_idx in tqdm(outer_folds.split(np.zeros(len(y)), y), desc="Outer Fold", total=outer_folds.get_n_splits(), ncols=80):
    train_dataset = Subset(dataset, train_idx)
    test_dataset = Subset(dataset, test_idx)
    
    best_val_score = -float('inf')
    best_params = None
    best_model = None

    # Inner loop for hyperparameter tuning
    for lr in tqdm(learning_rates, desc="Learning Rates", leave=False, ncols=80):
        for batch_size in tqdm(batch_sizes, desc="Batch Sizes", leave=False):
            val_scores = []

            for inner_train_idx, val_idx in tqdm(inner_folds.split(np.zeros(len(train_idx)), y[train_idx]), desc="Inner Fold", total=inner_folds.get_n_splits(), leave=False, ncols=80):
                inner_train_dataset = Subset(dataset, train_idx[inner_train_idx])
                val_dataset = Subset(dataset, train_idx[val_idx])
                
                train_loader = DataLoader(inner_train_dataset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                
                model = CustomResNetModel(base_model, num_classes)
                criterion = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(
                    filter(lambda p: p.requires_grad, model.parameters()), lr=lr
                )  
                
                model.train()
                for epoch in range(5): 
                    print(f"Epoch {epoch + 1}/5, Learning Rate: {lr}, Batch Size: {batch_size}")
                    epoch_loss = 0.0
                    for inputs, labels in train_loader:
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()
                        epoch_loss += loss.item()
                    
                    print(f"  Training Loss: {epoch_loss / len(train_loader):.4f}")
                
                model.eval()
                all_preds = []
                all_labels = []
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        outputs = model(inputs)
                        preds = torch.argmax(outputs, dim=1)
                        all_preds.append(preds)
                        all_labels.append(labels)
                
                all_preds = torch.cat(all_preds)
                all_labels = torch.cat(all_labels)
                val_accuracy = accuracy_score(all_labels.cpu(), all_preds.cpu())
                val_scores.append(val_accuracy)
            
            avg_val_score = np.mean(val_scores)
            
            if avg_val_score > best_val_score:
                best_val_score = avg_val_score
                best_params = {"lr": lr, "batch_size": batch_size}
                best_model = model
    
    # Test the best model on the outer test set
    test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False)
    best_model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = best_model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.append(preds)
            all_labels.append(labels)
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    test_accuracy = accuracy_score(all_labels.cpu(), all_preds.cpu())
    outer_results.append(test_accuracy)

    print(f"Fold Test Accuracy: {test_accuracy:.4f} | Best Params: {best_params}")

print(f"Nested CV Test Accuracy: {np.mean(outer_results):.4f} ± {np.std(outer_results):.4f}")


Learning Rates:   0%|                                     | 0/3 [00:00<?, ?it/s]

Batch Sizes:   0%|                                                                                                                                             | 0/2 [00:00<?, ?it/s]


Inner Fold:   0%|                                         | 0/2 [00:00<?, ?it/s]

Epoch 1/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.7469
Epoch 2/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.6960
Epoch 3/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.9394
Epoch 4/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 1.3071
Epoch 5/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.7132





Inner Fold:  50%|████████████████                | 1/2 [02:29<02:29, 149.05s/it]

Epoch 1/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.6169
Epoch 2/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.6699
Epoch 3/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.7225
Epoch 4/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 1.4705
Epoch 5/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.7360





Inner Fold: 100%|████████████████████████████████| 2/2 [04:48<00:00, 143.19s/it]


                                                                                

Batch Sizes:  50%|██████████████████████████████████████████████████████████████████                                                                  | 1/2 [04:48<04:48, 288.14s/it]


Inner Fold:   0%|                                         | 0/2 [00:00<?, ?it/s]

...

Fold Test Accuracy: 0.5000 | Best Params: {'lr': 0.001, 'batch_size': 16}



Learning Rates:   0%|                                     | 0/3 [00:00<?, ?it/s]

Batch Sizes:   0%|                                                                                                                                             | 0/2 [00:00<?, ?it/s]


Inner Fold:   0%|                                         | 0/2 [00:00<?, ?it/s]

Epoch 1/5, Learning Rate: 0.001, Batch Size: 16
  Training Loss: 0.7596
Epoch 2/5, Learning Rate: 0.001, Batch Size: 16





                                                                                

                                                                                                                                                                                     
Outer Fold:  50%|███████████████▌               | 1/2 [28:01<28:01, 1681.95s/it]


KeyboardInterrupt: 